# Read Offline from CSV

## Overview
Data manually downloaded from

 - **WSJ.com**
  
 - **Investing.com**

  
### Offline Data

| No | Symbol | Index Name | Source | URL Address |
| :--- | :----- | :----- | :---- | :----- |
| 1 | GC | gold_futures | investing | [Gold Futures Historical Data](https://www.investing.com/commodities/gold-historical-data) |
| 2 | DXY | usd_index | investing | [US Dollar Index Historical Data](https://www.investing.com/indices/usdollar-historical-data) |
| 3 | HG | copper_futures | investing | [Copper Futures Historical Data](https://www.investing.com/commodities/copper-historical-data) |
| 4 | COMP | nasdaq_comp | WSJ | [NASDAQ Composite Index](https://www.wsj.com/market-data/quotes/index/COMP/historical-prices) |
| 5 | DJIA | dow_jones | WSJ | [Dow Jones Industrial Average](https://www.wsj.com/market-data/quotes/index/DJIA/historical-prices) |
| 6 | SPX | sp500 | WSJ | [S&P 500 Index](https://www.wsj.com/market-data/quotes/index/SPX/historical-prices) |

### Data fields

WSJ: `['Date', 'Open', 'High', 'Low', 'Close']`

Investing:`['Date', 'Price', 'Open','High','Low','Vol','Change']`

## Import Libraries

In [1]:
# import necessary libraries
import pandas as pd
import os
import glob

import pickle

## Custom functions

In [2]:
def monthly_average(df):
    '''
    Get a dataframe with daily records 
    and return a new df with monthly average
    '''
    
    df['month'] = pd.DatetimeIndex(df.index).month
    df['year'] = pd.DatetimeIndex(df.index).year
    
    dfg = df.groupby(['year','month'], as_index=False).mean()
    dfg['Date'] = pd.to_datetime(dict(year = dfg.year, month = dfg.month, day = 1))
    
    dfg = dfg.drop(columns = ['month','year'])
    
    dfg.set_index('Date', drop=True, inplace=True)
    
    return dfg

## Set Directories

In [3]:
# Change directory one step back and save as the root directory
ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Change direcotry to the 'data\raw' folder
path = ROOT_DIR + '\\data\\raw\\'
print(path)

d:\gitProjects\WTI_Crude_Oil_Price_Prediction_Using_ML\data\raw\


In [4]:
# use glob to get all the csv files in the folder
csv_files = glob.glob(os.path.join(path, "*.csv"))
#dispaly csv file list
display(csv_files)

['d:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\copper_futures.csv',
 'd:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\dow_jones.csv',
 'd:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\gold_futures.csv',
 'd:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\nasdaq_comp.csv',
 'd:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\sp500.csv',
 'd:\\gitProjects\\WTI_Crude_Oil_Price_Prediction_Using_ML\\data\\raw\\usd_index.csv']

## Load all CSV files

In [5]:
def get_file_name(file_name):
    '''
    Get full file address and return the name
    '''
    name_ext = file_name.split("\\")[-1]
    name = name_ext.split(".")[0]
    
    return name

In [6]:
# Initialize list; this is the final list that will store all the data from the json pull. 
read_data = []

# loop over the list of csv files
for f in csv_files:
    
    # read the csv file    
    df = pd.read_csv(f, sep = ',', thousands=',')
        
    if ' Close' in df.columns:
        #rename column name
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close']
        value = 'Close'
        format_date = '%m/%d/%y'
        
        
    if 'Price' in df.columns:
        df.columns = ['Date', 'Price', 'Open','High','Low','Vol','Change']
        value = 'Price'
        format_date = '%m/%d/%Y'
    
    #select columns
    df = df[['Date', value]]

    #change data type to date and numeric
    df['value'] = pd.to_numeric(df[value])
    df['Date'] = pd.to_datetime(df['Date'], format=format_date, errors="raise")
    
    df = df.drop(columns = [value])
    df = df.rename(columns = {'value': get_file_name(f)}, errors="raise")
    
    #set Date as index
    df.set_index('Date', drop=True, inplace=True)
    
    print('Sucess:', get_file_name(f))
    display(df.head())
    
    read_data.append(df)
    # print the success report
    


Sucess: copper_futures


,copper_futures
Date,
2023-07-01,3.7690
2023-06-01,3.7595
2023-05-01,3.6370
2023-04-01,3.8812
2023-03-01,4.0945


Sucess: dow_jones


,dow_jones
Date,
2023-07-03,34418.47
2023-06-30,34407.60
2023-06-29,34122.42
2023-06-28,33852.66
2023-06-27,33926.74


Sucess: gold_futures


...

,nasdaq_comp
Date,
2023-07-03,13816.77
2023-06-30,13787.92
2023-06-29,13591.33
2023-06-28,13591.75
2023-06-27,13555.67


Sucess: sp500


,sp500
Date,
2023-07-03,4455.59
2023-06-30,4450.38
2023-06-29,4396.44
2023-06-28,4376.86
2023-06-27,4378.41


Sucess: usd_index


,usd_index
Date,
2023-07-01,103.15
2023-06-01,102.91
2023-05-01,104.33
2023-04-01,101.66
2023-03-01,102.51


## Data

### Print the features and sizes

In [7]:
for i in range(len(read_data)):
    name = read_data[i].columns[0]
    size = len(read_data[i])
    phrase = str(i) + ': '+ name + ' (1*' + str(size) + ')'
    print(phrase)

0: copper_futures (1*283)
1: dow_jones (1*5912)
2: gold_futures (1*283)
3: nasdaq_comp (1*5912)
4: sp500 (1*5912)
5: usd_index (1*283)


### Merge_order all data frames

In [8]:
df = read_data[0]
dfg = monthly_average(df)
dfg.index = pd.to_datetime(dfg.index, format = '%Y%m%d',errors='coerce')

for df_temp in read_data[1:]:
    dfg_temp = monthly_average(df_temp)
    dfg_temp.index = pd.to_datetime(dfg_temp.index, format = '%Y%m%d',errors='coerce')
    dfg = pd.merge_ordered(dfg, dfg_temp, on = 'Date')

## Check the final dataframe

In [9]:
display(dfg.round(2))

,Date,copper_futures,dow_jones,gold_futures,nasdaq_comp,sp500,usd_index
0,2000-01-01,0.85,11281.26,286.20,4013.49,1425.59,105.13
1,2000-02-01,0.80,10541.93,295.40,4410.87,1388.87,105.92
2,2000-03-01,0.81,10483.39,281.40,4802.99,1442.21,105.44
3,2000-04-01,0.80,10944.31,276.00,3863.64,1461.36,110.14
4,2000-05-01,0.82,10580.27,274.80,3528.42,1418.48,108.74
...,...,...,...,...,...,...,...
278,2023-03-01,4.09,32483.48,2004.10,11637.09,3968.56,102.51
279,2023-04-01,3.88,33731.28,2018.30,12073.46,4121.47,101.66
280,2023-05-01,3.64,33316.80,1982.10,12446.53,4146.17,104.33
281,2023-06-01,3.76,33904.98,1929.40,13463.78,4345.37,102.91


## Store the data

In [10]:
# Change direcotry to the 'data\raw' folder
save_dir = ROOT_DIR + '\\data\\interim\\'

# Set a file name
file_save = save_dir + 'offline_data.csv'
dfg.to_csv(file_save)

print('Save:\n', file_save)

Save:
 d:\gitProjects\WTI_Crude_Oil_Price_Prediction_Using_ML\data\interim\offline_data.csv
